<a href="https://colab.research.google.com/github/peeyushsinghal/da/blob/main/mitigating_bias_sa_da.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [42]:
! pip install ekphrasis # library to pre process twitter data
! pip install emoji --upgrade #library to deal with emoji data

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [125]:
## Import statements
import pandas as pd
import os
import torch
from torch.utils.data import Dataset, DataLoader
from torchtext.vocab import GloVe
import numpy as np
from ekphrasis.classes.preprocessor import TextPreProcessor
from ekphrasis.classes.tokenizer import SocialTokenizer
from ekphrasis.dicts.emoticons import emoticons
import emoji
from tqdm import tqdm
import random
import torch.optim as optim


In [44]:
# checking device
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Running on:{}".format(DEVICE))

Running on:cpu


# Data Handling

Mounting google drive for data in there

In [45]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


##Data configuration

In [46]:
BASE_PATH = '/content/drive/MyDrive/semeval-2018'
DATA_DIR = os.path.join(BASE_PATH,'datasets')

In [47]:
class TASK1(object):
  
    EI_reg = {
        'anger': {
            'train': os.path.join(
                DATA_DIR, 'task1/EI-reg/training/EI-reg-En-anger-train.txt'),
            'dev': os.path.join(
                DATA_DIR, 'task1/EI-reg/development/2018-EI-reg-En-anger-dev.txt'),
            'gold': os.path.join(
                DATA_DIR, 'task1/EI-reg/test-gold/2018-EI-reg-En-anger-test-gold.txt')
                }
        }

    V_reg = {
        'train': os.path.join(
            DATA_DIR, 'task1/V-reg/2018-Valence-reg-En-train.txt'),
        'dev': os.path.join(
            DATA_DIR, 'task1/V-reg/2018-Valence-reg-En-dev.txt'),
        'gold': os.path.join(
            DATA_DIR, 'task1/V-reg/2018-Valence-reg-En-test-gold.txt')
             }

    EEC = {
        'eec': os.path.join(
            DATA_DIR, 'task1/Equity-Evaluation-Corpus/Equity-Evaluation-Corpus.csv')
             }

##Dataloading and Parsing

Parsing regression data : `format [ID	Tweet	Affect Dimension	Intensity Score]`

In [48]:
def parse_reg(data_file, label_format='tuple')-> pd.DataFrame:
    """
    This is for datasets for the EI-reg and V-reg English tasks 
    Returns:
        df: dataframe with columns in the first row of file [ID-Tweet-Affect Dimension-Intensity Score]
    """
    with open(data_file, 'r') as fd:
      data = [l.strip().split('\t') for l in fd.readlines()]
    
    df = pd.DataFrame (data[1:],columns=data[0])
    return df

In [49]:
# def parse_reg(data_file, label_format='tuple')-> (list, list):
#     """
#     This is for datasets for the EI-reg and V-reg English tasks 
#     Returns:
#         X: a list of tweets
#         y: a list of (affect dimension, v) tuples corresponding to
#          the regression targets of the tweets
#     """
#     with open(data_file, 'r') as fd:
#         data = [l.strip().split('\t') for l in fd.readlines()][1:]
#     X = [d[1] for d in data]
#     y = [(d[2], float(d[3])) for d in data]
#     if label_format == 'list':
#         y = [l[1] for l in y]
#     return X, y

parsing EEC data : `format [ID	Sentence	Template	Person	Gender	Race Emotion	Emotion word]`

In [50]:
def parse_eec()->pd.DataFrame:
  """
  This is for EEC Dataset, it is a csv file
  Returns:
        df_eec: dataframe 
  """
  data_train = TASK1.EEC['eec']
  df_eec = pd.read_csv(data_train)
  return df_eec


In [51]:
def parse(task, dataset, emotion='anger') -> pd.DataFrame:
    if task == 'EI-reg':
        data_train = TASK1.EI_reg[emotion][dataset]
        df = parse_reg(data_train)
        df[df.columns[-1]] = df[df.columns[-1]].astype(float)
        return df
    elif task == 'V-reg':
        data_train = TASK1.V_reg[dataset]
        df = parse_reg(data_train)
        df[df.columns[-1]] = df[df.columns[-1]].astype(float)
        return df
    else:
        return None, None

In [52]:
## Creating Dataframes
df_EI_reg_train = parse('EI-reg','train')
df_EI_reg_val = parse('EI-reg','dev')
df_EI_reg_test = parse('EI-reg','gold')
df_V_reg_train = parse('V-reg','train')
df_V_reg_val = parse('V-reg','dev')
df_V_reg_test = parse('V-reg','gold')

dict_df= {'df_EI_reg_train':df_EI_reg_train, 
          'df_EI_reg_val':df_EI_reg_val, 
          'df_EI_reg_test':df_EI_reg_test, 
          'df_V_reg_train': df_V_reg_train, 
          'df_V_reg_val':df_V_reg_val, 
          'df_V_reg_test': df_V_reg_test 
          }

##PreProcess Twitter Data

In [53]:
# reference : https://github.com/cbaziotis/ekphrasis


text_processor = TextPreProcessor(
    # terms that will be normalized
    normalize=['url', 'email', 'percent', 'money', 'phone', 'user',
        'time', 'url', 'date', 'number'],
    # terms that will be annotated
    annotate={"hashtag", "allcaps", "elongated", "repeated",
        'emphasis', 'censored'},
    fix_html=True,  # fix HTML tokens
    
    # corpus from which the word statistics are going to be used 
    # for word segmentation 
    segmenter="twitter", 
    
    # corpus from which the word statistics are going to be used 
    # for spell correction
    corrector="twitter", 
    
    unpack_hashtags=True,  # perform word segmentation on hashtags
    unpack_contractions=True,  # Unpack contractions (can't -> can not)
    spell_correct_elong=False,  # spell correction for elongated words
    
    # select a tokenizer. You can use SocialTokenizer, or pass your own
    # the tokenizer, should take as input a string and return a list of tokens
    tokenizer=SocialTokenizer(lowercase=True).tokenize,
    
    # list of dictionaries, for replacing tokens extracted from the text,
    # with other expressions. You can pass more than one dictionaries.
    dicts=[emoticons]
)

Reading twitter - 1grams ...
Reading twitter - 2grams ...
Reading twitter - 1grams ...


In [54]:
# #### Example checks of pre-processing
# sentences = [
#     "CANT WAIT for the new season of #TwinPeaks ＼(^o^)／!!! #davidlynch #tvseries :)))",
#     "I saw the new #johndoe movie and it suuuuucks!!! WAISTED $10... #badmovies :/",
#     "@SentimentSymp:  can't wait for the Nov 9 #Sentiment talks!  YAAAAAAY !!! :-D http://sentimentsymposium.com/.",
#     "@MGBarbieri @SpalkTalk a@b.com And just saw your LinkedIn comment after I sent this! Thanks for the message :) 😀",
#     "💙💛🏆 @GeorgeePitman Young Player of The Season 🏆💛💙 #irony #actuallyseventy"
# ]

# for s in sentences:
#     print(" ".join(text_processor.pre_process_doc(s)))
# # print ([text_processor.pre_process_doc(s) for s in sentences])

In [55]:
def preprocess_tweets(df)-> pd.DataFrame:
  tweets = df.Tweet.to_list()
  # df['TweetTokens'] = [emoji.demojize(text_processor.pre_process_doc(tweet),language = 'en') for tweet in tweets] # Translates emoji in to word and preprocesss
  # df['TweetTokens'] = [text_processor.pre_process_doc(tweet) for tweet in tweets] # preprocesss
  # tweets_processed = [text_processor.pre_process_doc(tweet) for tweet in tweets] # preprocesss
  # for tweet in tweets_processed:
  #   for index, token in enumerate(tweet):
  #     if emoji.is_emoji(token):
  #       tweet[index] = emoji.demojize(token, language = 'en')

  tweets_processed = [" ".join(text_processor.pre_process_doc(tweet)) for tweet in tweets] # preprocesss
  # print (tweets_processed)
  for index, tweet in enumerate(tweets_processed):
      tweets_processed[index] = emoji.demojize(tweet, language = 'en')
  
  df['TweetTokens'] = tweets_processed
  # print(df)
  return df

In [56]:
# df_EI_reg_train = preprocess_tweets(df_EI_reg_train)
# df_V_reg_train = preprocess_tweets(df_V_reg_train)

for name, df in dict_df.items():
  df = preprocess_tweets(df)


In [57]:
# df_V_reg_train

TODO : 
* remove stop words
* stem
* lemmetize


In [58]:
df_V_reg_train.columns

Index(['ID', 'Tweet', 'Affect Dimension', 'Intensity Score', 'TweetTokens'], dtype='object')

In [59]:
def subset_df(df):
  return df[['TweetTokens','Intensity Score']]

In [60]:
dict_df_subset ={name+"_subset": subset_df(df) for name, df in dict_df.items() }

In [61]:
print (dict_df_subset)

{'df_EI_reg_train_subset':                                             TweetTokens  Intensity Score
0     <user> <user> shut up hashtags are cool <hasht...            0.562
1     it makes me so fucking irate jesus . nobody is...            0.750
2     lol adam the bull with his fake outrage . <rep...            0.417
3     <user> passed away early this morning in a fas...            0.354
4     <user> lol wow i was gonna say really ? ! <rep...            0.438
...                                                 ...              ...
1696  got a <money> tip from a drunk uber passenger ...            0.708
1697  <user> <user> <user> <user> fucker blocked me ...            0.625
1698                                <user> i look rabid            0.472
1699  <user> i am not surprised , i would be fuming ...            0.479
1700  <user> the pout tips me over the edge . i am m...            0.490

[1701 rows x 2 columns], 'df_EI_reg_val_subset':                                            Twee

## Creating Pytorch Datasets

### Creating Vocabulary
Before we create the Dataset, we need to define a process to build our vocabulary. For this,
We’ll create a “Vocabulary” class which will create the word-to-index and index-to-word mappings using only the train dataframe we created before
Also, the “Vocabulary” class returns the numericalized version of each sentence in our dataframe. Eg: [‘i’, ‘love’, ‘apple’] -> [23, 54, 1220]. We need to convert the words to numbers as models expect each word in our vocabulary to be represented by a number

In [62]:
#######################################################
#               Define Vocabulary Class
#######################################################

class Vocabulary:
  
    '''
    __init__ method is called by default as soon as an object of this class is initiated
    we use this method to initiate our vocab dictionaries
    '''
    def __init__(self, freq_threshold = 3, max_size = 10000):
        '''
        freq_threshold : the minimum times a word must occur in corpus to be treated in vocab
        max_size : max source vocab size. Eg. if set to 10,000, we pick the top 10,000 most frequent words and discard others
        '''
        #initiate the index to token dict
        ## <PAD> -> padding, used for padding the shorter sentences in a batch to match the length of longest sentence in the batch
        ## <UNK> -> words which are not found in the vocab are replace by this token
        # self.itos = {0: '<PAD>', 1: '<UNK>', 2:'<NUMBER>', 3: '<CURRENCY>', 4: '<URL>'}
        self.itos = {0: '<PAD>', 1: '<UNK>'}
        
        
        #initiate the token to index dict
        self.stoi = {k:j for j,k in self.itos.items()}
        self.original_stoi = self.stoi.copy()
#         print(self.stoi)
        
        self.freq_threshold = freq_threshold
        self.max_size = max_size
    
    '''
    __len__ is used by dataloader later to create batches
    '''
    def __len__(self):
        return len(self.itos)
    
    '''
    a simple tokenizer to split on space and converts the sentence to list of words
    '''
    @staticmethod
    def tokenizer(text):
#         return [tok.strip() for tok in text.split(' ')]
        return [tok.lower().strip() for tok in text.split(' ')] # this is commented out to avoid <NUMBER> ,<UNK> lowering
#         return [tok.lower().strip() for tok in text.split(' ') if tok not in list(self.stoi.keys())] 
    
    '''
    build the vocab: create a dictionary mapping of index to string (itos) and string to index (stoi)
    output ex. for stoi -> {'the':6, 'a':7, 'an':8}
    '''
    def build_vocabulary(self, sentence_list):
        #calculate the frequencies of each word first to remove the words with freq < freq_threshold
#         frequencies = {}  #init the freq dict
        frequencies = {k:self.max_size+1 for _,k in self.itos.items()}  # updated so that intial ones are also part of this
        
        # idx = 5 #index from which we want our dict to start. We already used 4 indexes for pad, unk...
        idx = len(self.original_stoi)
        
        #calculate freq of words
        for sentence in sentence_list:
            list_word = [tok.lower().strip() for tok in sentence.split(' ') if tok not in list(self.stoi.keys())] 
            for word in list_word:
#             for word in self.tokenizer(sentence):
                
                if word not in frequencies.keys():
                    frequencies[word]=1
                else:
                    
                    frequencies[word]+=1
                    
#         print ("----2-----\n",frequencies)
        
        #limit vocab by removing low freq words
        frequencies = {k:v for k,v in frequencies.items() if v>self.freq_threshold} 
        
#         print ("----3-----\n",frequencies)
        
        #limit vocab to the max_size specified
        frequencies = dict(sorted(frequencies.items(), key = lambda x: -x[1])[:self.max_size-idx]) # idx =5 for pad, unk...
        
#         print ("----4-----\n",frequencies)
            
        #create vocab
        for key in set(self.stoi.keys()):
            frequencies.pop(key)
        
#         print ("----5-----\n",frequencies)
        
        for word in frequencies.keys():
            self.stoi[word] = idx
            self.itos[idx] = word
            idx+=1
        
#         print ("----6-----\n",self.stoi)
        
    '''
    convert the list of words to a list of corresponding indexes
    '''    
    def numericalize(self, text):
        #tokenize text
#         tokenized_text = self.tokenizer(text)
#         print("---------\n",self.original_stoi.keys())
        tokenized_text = []
        for tok in text.split(' '):
            if tok not in list(self.original_stoi.keys()):
                tokenized_text.append(tok.lower().strip())
            else:
                tokenized_text.append(tok.strip())
                
#         tokenized_text = [tok.lower().strip() for tok in text.split(' ') if tok not in list(self.original_stoi.keys())]
        numericalized_text = []
        for token in tokenized_text:
            if token in self.stoi.keys():
                numericalized_text.append(self.stoi[token])
            else: #out-of-vocab words are represented by UNK token index
                numericalized_text.append(self.stoi['<UNK>'])
                
        return numericalized_text

In [63]:
# # #create a vocab class with freq_threshold=0 and max_size=100
# voc = Vocabulary(0, 100)
# sentence_list = ['that is a cat CAT', 'that is not a dog']
# #build vocab
# voc.build_vocabulary(sentence_list)

# print('index to string: ',voc.itos)
# print('string to index:',voc.stoi)

# print('numericalize -> cat and a dog <URL>: ', voc.numericalize('cat and a dog <NUMBER>'))

In [64]:
# Creating sentence list of all the training dataframes to create vocabulary later, this would mean a more robust vocab
sentence_list = []
for name,df in dict_df_subset.items():
  if "train" in name:
    sentence_list.extend(df.TweetTokens.to_list())
print(len(sentence_list))

2882


In [65]:
freq_threshold = 3
vocab_max_size = 50000

vocab = Vocabulary(freq_threshold, vocab_max_size)
vocab.build_vocabulary(sentence_list)

### Build Train_Dataset
We first inherit PyTorch's Dataset class.
Then, we initialize and build the vocabs for subject in our train data frame.
Then, we use the getitem() method to numericalize the subject 1 example at a time for the data loader (a function to load data in batches).

In [66]:
#######################################################
#               Define Train_Dataset class
#######################################################

class Train_Dataset(Dataset):
    '''
    Initiating Variables
    df: the training dataframe
    subject : the name of target text column in the dataframe
    transform : If we want to add any augmentation
    freq_threshold : the minimum times a word must occur in corpus to be treated in vocab
    vocab_max_size : max  vocab size
    '''
    
    def __init__(self, df, subject, label_col, vocab , max_sentence_length = 150, transform=None, freq_threshold = 5,
                vocab_max_size = 50000):
    
        self.df = df
        self.transform = transform
        
        #get body and label
        self.subject_texts = self.df[subject]
        self.labels = self.df[label_col]
        self.vocab = vocab
        
        # ##VOCAB class has been created above
        # #Initialize vocab object and build vocabulary
        # self.vocab = Vocabulary(freq_threshold, vocab_max_size)
        # self.vocab.build_vocabulary(self.subject_texts.tolist())
        self.max_sentence_length = max_sentence_length
        
    def __len__(self):
        return len(self.df)
    
    '''
    __getitem__ runs on 1 example at a time. Here, we get an example at index and return its numericalize source and
    target values using the vocabulary objects we created in __init__
    '''
    def __getitem__(self, index):
        subject_text = self.subject_texts[index]
        label = self.labels[index]
#         print(subject_text)
        
        if self.transform is not None:
            subject_text = self.transform(subject_text)
            
        #numericalize texts ['<SOS>','cat', 'in', 'a', 'bag','<EOS>'] -> [1,12,2,9,24,2]
        numerialized_subject =[]
        numerialized_subject += self.vocab.numericalize(subject_text)
        
        while len(numerialized_subject) < self.max_sentence_length:
            numerialized_subject.append(0)
        
        #convert the list to tensor and return
        return torch.tensor(numerialized_subject[:self.max_sentence_length]),torch.tensor(label)
#         return torch.tensor(numerialized_subject[:self.train_dataset.max_sentence_length]),label

In [67]:
dict_df_subset.keys()

dict_keys(['df_EI_reg_train_subset', 'df_EI_reg_val_subset', 'df_EI_reg_test_subset', 'df_V_reg_train_subset', 'df_V_reg_val_subset', 'df_V_reg_test_subset'])

In [68]:
df_train = dict_df_subset['df_EI_reg_train_subset']
df_train

,TweetTokens,Intensity Score
0,<user> <user> shut up hashtags are cool <hasht...,0.562
1,it makes me so fucking irate jesus . nobody is...,0.750
2,lol adam the bull with his fake outrage . <rep...,0.417
3,<user> passed away early this morning in a fas...,0.354
4,<user> lol wow i was gonna say really ? ! <rep...,0.438
...,...,...
1696,got a <money> tip from a drunk uber passenger ...,0.708
1697,<user> <user> <user> <user> fucker blocked me ...,0.625
1698,<user> i look rabid,0.472
1699,"<user> i am not surprised , i would be fuming ...",0.479


In [70]:
dict_train_dataset ={}
for name,df in dict_df_subset.items():
  if "train" in name:
    dataset_name = name+"_dataset"
    # vars()[dataset_name] = Train_Dataset(df,'TweetTokens','Intensity Score', max_sentence_length =200) # dynamically assigning datasetname
    dict_train_dataset[dataset_name] = Train_Dataset(df,'TweetTokens','Intensity Score', vocab, max_sentence_length =200) # dynamically assigning datasetname

In [72]:
print(dict_train_dataset.keys())

dict_keys(['df_EI_reg_train_subset_dataset', 'df_V_reg_train_subset_dataset'])


In [73]:
# i = random.randint(0,len(dict_train_dataset['df_EI_reg_train_subset_dataset']))
# print(dict_df_subset['df_EI_reg_train_subset'].loc[i][['TweetTokens','Intensity Score']])
# print((dict_train_dataset['df_EI_reg_train_subset_dataset'][i][0]))
# print((dict_train_dataset['df_EI_reg_train_subset_dataset'][i][1]))

In [75]:
train_dataset_obj = Train_Dataset(df_train,'TweetTokens','Intensity Score', vocab,max_sentence_length =200)

In [76]:
# i = random.randint(0,len(train_dataset_obj))
# print(df_train.loc[i][['TweetTokens','Intensity Score']])
# print((train_dataset_obj[i][1]))
# print(len(train_dataset_obj[i][0]))
# print(train_dataset_obj[i][0])

In [77]:
# print(df_EI_reg_train_subset_dataset)

In [78]:
# i = random.randint(0,len(df_EI_reg_train_subset_dataset))
# # print(train.loc[i][['body','label']])
# print(type(df_EI_reg_train_subset_dataset[i][1]))
# len(df_EI_reg_train_subset_dataset[i][0])

### Build Validation Dataset

In [79]:
#######################################################
#               Define Dataset Class
#######################################################

class Validation_Dataset(Dataset):
    def __init__(self, train_dataset, df, subject, label_col, transform = None):
        self.df = df
        self.transform = transform
        
        #train dataset will be used as lookup for vocab
        self.train_dataset = train_dataset
        
        #get body and label
        self.subject_texts = self.df[subject]
        self.labels = self.df[label_col]
    
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self,index):
        
        subject_text = self.subject_texts[index]
        label = self.labels[index]
#         print(subject_text)
        
        if self.transform is not None:
            subject_text = self.transform(subject_text)
            
            

        #numericalize texts ['cat', 'in', 'a', 'bag'] -> [12,2,9,24]
        numerialized_subject = []
        numerialized_subject += self.train_dataset.vocab.numericalize(subject_text)
#         print("max sentence length", self.train_dataset.max_sentence_length)
        while len(numerialized_subject) < self.train_dataset.max_sentence_length:
            numerialized_subject.append(0)
            
        #convert the list to tensor and return
#         return torch.tensor(numerialized_subject),label

#         #convert the list to tensor and return
        return torch.tensor(numerialized_subject[:self.train_dataset.max_sentence_length]),torch.tensor(label)


In [80]:
dict_val_dataset = {}
for name,df in dict_df_subset.items():
  if "train" in name:
    train_dataset_name = name+"_dataset"
    val_df_name = name[:len(name)-13]+"_val_subset"
    val_dataset_name = val_df_name + "_dataset"
    dict_val_dataset[val_dataset_name] = Validation_Dataset(dict_train_dataset[train_dataset_name], dict_df_subset[val_df_name], 'TweetTokens','Intensity Score', transform = None)

In [81]:
dict_val_dataset.keys()

dict_keys(['df_EI_reg_val_subset_dataset', 'df_V_reg_val_subset_dataset'])

In [82]:
# i = random.randint(0,len(dict_val_dataset['df_EI_reg_val_subset_dataset']))
# # i = 3774
# print("i=",i)
# print(dict_df_subset['df_EI_reg_val_subset'].loc[i][['TweetTokens','Intensity Score']])

# print((dict_val_dataset['df_EI_reg_val_subset_dataset'][i][0]))
# print((dict_val_dataset['df_EI_reg_val_subset_dataset'][i][1]))

## Create Dataloader


In [83]:

#######################################################
#            Define Dataloader Functions
#######################################################

# If we run a next(iter(data_loader)) we get an output of batch_size * (num_workers+1)
def get_loader(dataset, batch_size, num_workers=1, shuffle=True, pin_memory=True): #increase num_workers according to CPU
    loader = DataLoader(dataset, batch_size = batch_size, num_workers = num_workers,
                        shuffle=shuffle,
                       pin_memory=pin_memory)
    
    return loader


In [84]:
dict_train_loader = {}
batch_size = 8
for name, dataset in dict_train_dataset.items():
  name_dataloader = name+"_dataloader"
  dict_train_loader[name_dataloader] = get_loader(dataset, batch_size)
  x = next(iter(dict_train_loader[name_dataloader]))
  print(name_dataloader, x[0].shape, x[1].shape, type(x[0]), type (x[1]))

df_EI_reg_train_subset_dataset_dataloader torch.Size([8, 200]) torch.Size([8]) <class 'torch.Tensor'> <class 'torch.Tensor'>
df_V_reg_train_subset_dataset_dataloader torch.Size([8, 200]) torch.Size([8]) <class 'torch.Tensor'> <class 'torch.Tensor'>


In [85]:
dict_val_loader = {}
batch_size = 8 
for name, dataset in dict_val_dataset.items():
  name_dataloader = name+"_dataloader"
  dict_val_loader[name_dataloader] = get_loader(dataset,batch_size)
  x = next(iter(dict_val_loader[name_dataloader]))
  print(name_dataloader, x[0].shape, x[1].shape, type(x[0]), type (x[1]))


df_EI_reg_val_subset_dataset_dataloader torch.Size([8, 200]) torch.Size([8]) <class 'torch.Tensor'> <class 'torch.Tensor'>
df_V_reg_val_subset_dataset_dataloader torch.Size([8, 200]) torch.Size([8]) <class 'torch.Tensor'> <class 'torch.Tensor'>


In [86]:
next(iter(dict_val_loader['df_EI_reg_val_subset_dataset_dataloader'] ))

[tensor([[598,  43,   1,  ...,   0,   0,   0],
         [  1,   1,  87,  ...,   0,   0,   0],
         [  5,  33, 185,  ...,   0,   0,   0],
         ...,
         [  5,   1,  78,  ...,   0,   0,   0],
         [ 25, 655,   4,  ...,   0,   0,   0],
         [  5,  78, 337,  ...,   0,   0,   0]]),
 tensor([0.2810, 0.5160, 0.6450, 0.3770, 0.6560, 0.3330, 0.4060, 0.4700],
        dtype=torch.float64)]

# Embeddings 

In [87]:
# # Creating sentence list of all the training dataframes to create vocabulary later, this would mean a more robust vocab
# sentence_list = []
# for name,df in dict_df_subset.items():
#   if "train" in name:
#     sentence_list.extend(df.TweetTokens.to_list())
# print(len(sentence_list))

In [88]:
# freq_threshold = 3
# vocab_max_size = 50000

# vocab = Vocabulary(freq_threshold, vocab_max_size)
# vocab.build_vocabulary(sentence_list)

In [89]:
len(vocab.stoi)

1619

In [90]:
global_vectors = GloVe(name='840B', dim=300)


In [91]:

def load_pretrained_vectors(word2idx, embedding_name = 'glove', embedding_file = global_vectors):
    """Load pretrained vectors and create embedding layers.
    
    Args:
        word2idx - vocab.stoi (Dict): Vocabulary built from the corpus
        embedding_name (str): the type of embedding - glove for GloVe or word2vec for word2vec
        embedding_file (object) :optional embedding file

    Returns:
        embeddings (np.array): Embedding matrix with shape (N, d) where N is
            the size of word2idx and d is embedding dimension
    """

    if embedding_name == 'glove':
      print("Loading pretrained vectors...")
      if embedding_file:
        global_vectors = embedding_file
      else:
        global_vectors = GloVe(name='840B', dim=300)

      print("Processing pretrained vectors...")
      d = 300
      print("\ndimension of pretained embedding: ", d)

      # Initilize random embeddings
      embeddings = np.random.uniform(-0.25, 0.25, (len(word2idx), d))
      embeddings[word2idx['<PAD>']] = np.zeros((d,))
      
      # Load pretrained vectors
      count = 0 
      for word in global_vectors.stoi:
        if word in word2idx:
            count +=1
            embeddings[word2idx[word]] = global_vectors[word]
      print(f"There are {count} / {len(word2idx)} pretrained vectors found.")
        
      print("Process Completed...")
      return embeddings

    else:
      print(" Embedding not implemented, returning zero embedding")
      return np.zeros(len(word2idx), 300)
    
    
#     # downloaded word2vec from https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz

#     word2vec = models.KeyedVectors.load_word2vec_format(word2vec_path, binary=True)
    
#     print("Processing pretrained vectors...")
#     d = word2vec.vector_size
#     print("\ndimension of pretained embedding: ", d)
    
#     # Initilize random embeddings
#     embeddings = np.random.uniform(-0.25, 0.25, (len(word2idx), d))
#     embeddings[word2idx['<PAD>']] = np.zeros((d,))

#     # Load pretrained vectors
#     count = 0 
#     for word in word2vec.key_to_index:
#         if word in word2idx:
#             count +=1
#             embeddings[word2idx[word]] = word2vec.get_vector(word)
    
    
# #     count = 0
# #     for line in tqdm_notebook(fin):
# #         tokens = line.rstrip().split(' ')
# #         word = tokens[0]
# #         if word in word2idx:
# #             count += 1
# #             embeddings[word2idx[word]] = np.array(tokens[1:], dtype=np.float32)

#     print(f"There are {count} / {len(word2idx)} pretrained vectors found.")
    
#     print("Process Completed...")
#     return embeddings


In [92]:
pre_trained_embeddings = load_pretrained_vectors(vocab.stoi, embedding_name = 'glove', embedding_file = global_vectors)

Loading pretrained vectors...
Processing pretrained vectors...

dimension of pretained embedding:  300
There are 1548 / 1619 pretrained vectors found.
Process Completed...


In [98]:
print(pre_trained_embeddings.shape)

(1619, 300)


In [93]:
embedding_tensor = torch.tensor(pre_trained_embeddings)
embedding_tensor.shape

torch.Size([1619, 300])

# Model Creation

## Gradient Reversal Layer Function

In [94]:
from torch.autograd import Function

class ReverseLayerF(Function):

    @staticmethod
    def forward(ctx, x, alpha):
        ctx.alpha = alpha

        return x.view_as(x)

    @staticmethod
    def backward(ctx, grad_output):
        output = grad_output.neg() * ctx.alpha

        return output, None

## CNN 1-D Model

Reference: A Sensitivity Analysis of (and Practitioners' Guide to) Convolutional Neural Networks for Sentence Classification, Ye Zhang, Byron Wallace 2015

Difference: 

1.   use of embedding
2.   use of sigmoid function, as we are having a regression model not a classififer as the main task



In [102]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class CNN_SA(nn.Module):
    """An 1D Convulational Neural Network for Sentiment Analysis."""
    def __init__(self,
                 pretrained_embedding=None,
                 freeze_embedding=False,
                 vocab_size=None,
                 embed_dim=300,
                 filter_sizes=[2, 3, 4, 5],
                 num_filters=[ 100, 100, 100, 100],
                 num_classes=2,
                 dropout=0.25):
        """
        The constructor for CNN_SA class.

        Args:
            pretrained_embedding (torch.Tensor): Pretrained embeddings with
                shape (vocab_size, embed_dim)
            freeze_embedding (bool): Set to False to fine-tune pretraiend
                vectors. Default: False
            vocab_size (int): Need to be specified when not pretrained word
                embeddings are not used.
            embed_dim (int): Dimension of word vectors. Need to be specified
                when pretrained word embeddings are not used. Default: 300
            filter_sizes (List[int]): List of filter sizes. Default: [2, 3, 4, 5]
            num_filters (List[int]): List of number of filters, has the same
                length as `filter_sizes`. Default: [100, 100, 100, 100]
            n_classes (int): Number of classes (domain classification usage). 
            Default: 2
            dropout (float): Dropout rate. Default: 0.25
        """

        super(CNN_SA, self).__init__()
        
         #---------------------Feature Extractor Network----------------------#
        # Embedding layer
        if pretrained_embedding is not None:
            self.vocab_size, self.embed_dim = pretrained_embedding.shape
            self.embedding = nn.Embedding.from_pretrained(pretrained_embedding,
                                                          freeze=freeze_embedding)
        else:
            self.embed_dim = embed_dim
            self.embedding = nn.Embedding(num_embeddings=vocab_size,
                                          embedding_dim=self.embed_dim,
                                          padding_idx=0,
                                          max_norm=5.0)
        # Conv Network
        self.feature_extractor = nn.ModuleList([
            nn.Conv1d(in_channels=self.embed_dim,
                      out_channels=num_filters[i],
                      kernel_size=filter_sizes[i])
            for i in range(len(filter_sizes))
        ])
        
        #---------------------Regression Network------------------------#
        # Fully-connected layer and Dropout
        self.regression = nn.Sequential(
            nn.Dropout(p=dropout),
            nn.Linear(np.sum(num_filters), np.sum(num_filters)//2),
            nn.ReLU(),
#             nn.BatchNorm1d(np.sum(num_filters)//2),
            # nn.Linear(np.sum(num_filters)//2, num_classes), # for classification
            nn.Linear(np.sum(num_filters)//2, 1), # for regression
            # nn.LogSoftmax(dim=1) # for classification
            nn.Sigmoid() # for regession (values between 0 and 1)
        )
        
        #---------------------Domain Classifier Network------------------------#
        # Fully-connected layer and Dropout
        self.domain_classifier = nn.Sequential(
            nn.Dropout(p=dropout),
            nn.Linear(np.sum(num_filters), np.sum(num_filters)//2),
            nn.ReLU(),
#             nn.BatchNorm1d(np.sum(num_filters)//2),
            nn.Linear(np.sum(num_filters)//2, num_classes),
            nn.LogSoftmax(dim=1)
        )
        
        

    def forward(self, input_ids,alpha=1):
        """Perform a forward pass through the network.

        Args:
            input_ids (torch.Tensor): A tensor of token ids with shape
                (batch_size, max_sent_length)

        Returns:
            sigmoid (torch.Tensor) : Output sigmoid 
            logits (torch.Tensor): Output logits with shape (batch_size,
                n_classes)
        """

        # Get embeddings from `input_ids`. Output shape: (b, max_len, embed_dim)
#         input_ids = torch.tensor(input_ids).to(torch.int64)
        input_ids = input_ids.clone().detach().to(torch.int64)
#         print(input_ids.shape)
        
        x_embed = self.embedding((input_ids)).float()
#         print(x_embed.shape)


        # Permute `x_embed` to match input shape requirement of `nn.Conv1d`.
        # Output shape: (b, embed_dim, max_len)
        x_reshaped = x_embed.permute(0, 2, 1)
#         print(x_reshaped.shape)
        

#         # Apply CNN and ReLU. Output shape: (b, num_filters[i], L_out)
        x_conv_list = [F.relu(conv1d(x_reshaped)) for conv1d in self.feature_extractor]
#         print(x_conv_list[1].shape)

#         # Max pooling. Output shape: (b, num_filters[i], 1)
        x_pool_list = [F.max_pool1d(x_conv, kernel_size=x_conv.shape[2])
            for x_conv in x_conv_list]
#         print( x_pool_list[3].shape)
        
#         # Concatenate x_pool_list to feed the fully connected layer.
#         # Output shape: (b, sum(num_filters))
        x_feature = torch.cat([x_pool.squeeze(dim=2) for x_pool in x_pool_list],
                         dim=1)
#         print(x_feature.shape)
        
# #         # Compute logits. Output shape: (b, n_classes)
#         logits = self.fc(self.dropout(x_feature))
#         print(logits)

        reverse_feature = ReverseLayerF.apply(x_feature, alpha)
    
        regression_output = self.regression(x_feature)
    
        domain_classifier_output = self.domain_classifier(reverse_feature)
#         print(domain_classifier_logits.shape)

#         return logits
        return regression_output, domain_classifier_output


In [103]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
model = CNN_SA(pretrained_embedding=embedding_tensor,freeze_embedding=True).to(device)
print(model)

cpu
CNN_SA(
  (embedding): Embedding(1619, 300)
  (feature_extractor): ModuleList(
    (0): Conv1d(300, 100, kernel_size=(2,), stride=(1,))
    (1): Conv1d(300, 100, kernel_size=(3,), stride=(1,))
    (2): Conv1d(300, 100, kernel_size=(4,), stride=(1,))
    (3): Conv1d(300, 100, kernel_size=(5,), stride=(1,))
  )
  (regression): Sequential(
    (0): Dropout(p=0.25, inplace=False)
    (1): Linear(in_features=400, out_features=200, bias=True)
    (2): ReLU()
    (3): Linear(in_features=200, out_features=1, bias=True)
    (4): Sigmoid()
  )
  (domain_classifier): Sequential(
    (0): Dropout(p=0.25, inplace=False)
    (1): Linear(in_features=400, out_features=200, bias=True)
    (2): ReLU()
    (3): Linear(in_features=200, out_features=2, bias=True)
    (4): LogSoftmax(dim=1)
  )
)


In [111]:


for name, train_loader in dict_train_loader.items():
  tweet, intensity = next(iter(train_loader))
  print (name, "\n", tweet, intensity)
  a, b = model(tweet)
  print ("a - sigmoid output\n", a)
  print ("b - softmax output\n", b)

df_EI_reg_train_subset_dataset_dataloader 
 tensor([[ 53,  13,  40,  ...,   0,   0,   0],
        [  5,  17,  11,  ...,   0,   0,   0],
        [  5,   5,   1,  ...,   0,   0,   0],
        ...,
        [  5, 195,  48,  ...,   0,   0,   0],
        [  1, 429,  31,  ...,   0,   0,   0],
        [  5, 185,   9,  ...,   0,   0,   0]]) tensor([0.5830, 0.6460, 0.6670, 0.5210, 0.5830, 0.4580, 0.3540, 0.3960],
       dtype=torch.float64)
a - sigmoid output
 tensor([[0.5078],
        [0.5082],
        [0.5083],
        [0.5106],
        [0.4997],
        [0.5144],
        [0.5029],
        [0.5007]], grad_fn=<SigmoidBackward0>)
b - softmax output
 tensor([[-0.7902, -0.6047],
        [-0.7487, -0.6405],
        [-0.7778, -0.6151],
        [-0.7245, -0.6628],
        [-0.7425, -0.6461],
        [-0.7311, -0.6566],
        [-0.8051, -0.5924],
        [-0.8086, -0.5897]], grad_fn=<LogSoftmaxBackward0>)
df_V_reg_train_subset_dataset_dataloader 
 tensor([[   5,    5,    5,  ...,    0,    0,    0],
 

# Training and Test Function

## Typical Training, Test Function (without Domain Adaptation)

In [132]:
# Training Function

from tqdm import tqdm # for beautiful model training updates


def train_model(model, device, train_loader, optimizer, epoch):
    model.train() # setting the model in training mode
    pbar = tqdm(train_loader) # putting the iterator in pbara
    correct = 0 # for accuracy numerator
    processed =0 # for accuracy denominator
    current_loss = 0.0
    for batch_idx, batch in enumerate(pbar):

        tweets, intensities = batch[0].to(device), batch[1].float().to(device)  # plural, we are not interested in domain
        #sending data to CPU or GPU as per device

        optimizer.zero_grad() # setting gradients to zero to avoid accumulation

        y_preds, _ = model(tweets) # forward pass, result captured in y_preds (plural as there are many body in a batch)
        # we are not interested in domain prediction
        # the predictions are in one hot vector

        loss = F.mse_loss(y_preds,intensities) # Computing loss        

        train_losses.append(loss) # to capture loss over many epochs

        loss.backward() # backpropagation
        optimizer.step() # updating the params

        # preds = y_preds.argmax(dim=1, keepdim=True)  # get the index olf the max log-probability
        # correct += preds.eq(labels.view_as(preds)).sum().item()
        current_loss += loss.item()

        processed += len(tweets)

        pbar.set_description(desc= f'Loss={loss.item()} Batch_id={batch_idx} Average loss={100*current_loss/processed:0.2f}')
        current_loss = 0.0


In [134]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

typical_model =  CNN_SA(pretrained_embedding=embedding_tensor,
                  freeze_embedding=True,
                 ).to(device)


optimizer = optim.Adam(typical_model.parameters(), lr=0.001)

train_losses = [] # to capture train losses over training epochs
train_accuracy = [] # to capture train accuracy over training epochs
test_losses = [] # to capture test losses 
test_accuracy = [] # to capture test accuracy 

EPOCHS = 2
# EPOCHS = 10
for train_name, train_loader in dict_train_loader.items():
  for epoch in range(EPOCHS):
    print("EPOCH:", epoch+1)
    train_model(typical_model, device, train_loader, optimizer, epoch)
    print("for source validation.......")
    # val_name = train_name.replace("train", "val" )
    # test_model(typical_model, device, dict_val_loader[val_name])
    # print("for target validation.......")
    # test_model(typical_model, device, val_target_loader)
  break

print("----------------------training complete-----------------")
# torch.save(typical_model.state_dict(),"sentiment_analysis_typical.pt")

cpu
EPOCH: 1


  0%|          | 0/213 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:23: UserWarning: Using a target size (torch.Size([8])) that is different to the input size (torch.Size([8, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
Loss=0.021369850262999535 Batch_id=211 Average loss=0.00: 100%|█████████▉| 212/213 [00:19<00:00, 11.22it/s]/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:23: UserWarning: Using a target size (torch.Size([5])) that is different to the input size (torch.Size([5, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
Loss=0.03859928995370865 Batch_id=212 Average loss=0.00: 100%|██████████| 213/213 [00:19<00:00, 11.00it/s]


for source validation.......
EPOCH: 2


Loss=0.02852105163037777 Batch_id=212 Average loss=0.00: 100%|██████████| 213/213 [00:19<00:00, 11.14it/s]

for source validation.......
----------------------training complete-----------------
